In [ ]:
!pip install transformers

     |████████████████████████████████| 1.9MB 6.5MB/s 
     |████████████████████████████████| 3.2MB 46.4MB/s 
     |████████████████████████████████| 890kB 45.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6b6604bb3683a1e02cbd365aade615f716b04f8c40d3049d5645765e3cdfe2c1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
import transformers
from transformers import AutoModel, BertTokenizerFast, AutoTokenizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.nn as nn
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


# Load data and create token, sequence and mask

In [ ]:
train_df = pd.read_csv("/content/drive/My Drive/vlsp/clean_train_df_ext.csv")
test_df = pd.read_csv("/content/drive/My Drive/vlsp/clean_final_test_df.csv")

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  5172 non-null   int64  
 1   user_name           5172 non-null   object 
 2   timestamp_post      5172 non-null   float64
 3   num_like_post       5172 non-null   float64
 4   num_comment_post    5172 non-null   float64
 5   num_share_post      5172 non-null   float64
 6   label               5172 non-null   int64  
 7   clean_post_message  5102 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 323.4+ KB


In [ ]:
X = train_df['clean_post_message'].astype(str)
y = train_df['label']

In [ ]:
X_train , X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, shuffle=False)

In [ ]:
X_test = test_df['clean_post_message'].astype(str)

In [ ]:
max_seq_len = 50

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    X_train.tolist(),
    max_length=max_seq_len,
    pad_to_max_length=True,
    truncation=False,
    return_token_type_ids=False
)

tokens_val = tokenizer.batch_encode_plus(
    X_val.tolist(),
    max_length=max_seq_len,
    pad_to_max_length=True,
    truncation=False,
    return_token_type_ids=False
)

tokens_test = tokenizer.batch_encode_plus(
    X_test.tolist(),
    max_length=max_seq_len,
    pad_to_max_length=True,
    truncation=False,
    return_token_type_ids=False
)



Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(y_train.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(y_val.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])

In [ ]:
for param in phobert.parameters():
  param.require_grad = False

In [ ]:
with torch.no_grad():
  last_hidden_states = phobert(train_seq, attention_mask=train_mask, output_hidden_states=True)


In [ ]:
torch.save(last_hidden_states[2][-1][:,0, ...], 'train_emb.pt')

In [ ]:
with torch.no_grad():
  last_hidden_states_val = phobert(val_seq, attention_mask=val_mask, output_hidden_states=True)


In [ ]:
torch.save(last_hidden_states_val[2][-1][:,0, ...], 'val_emb.pt')

In [ ]:
with torch.no_grad():
  last_hidden_states_test = phobert(test_seq, attention_mask=test_mask, output_hidden_states=True)


In [ ]:
torch.save(last_hidden_states_test[2][-1][:,0, ...], 'test_emb.pt')

In [ ]:
train_features = torch.load('train_emb.pt').numpy() #last_hidden_states[0][:, 0, :].numpy()
val_features = torch.load('val_emb.pt').numpy() #last_hidden_states_val[0][:, 0, :].numpy()
test_features = torch.load('test_emb.pt').numpy() #last_hidden_states_test[0][:, 0, :].numpy()

In [ ]:
all_train = np.concatenate((train_features, val_features), axis=0)
all_y = np.concatenate((y_train, y_val))

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

In [ ]:
preds = []
history = []
for (i, (train_idx, test_idx)) in enumerate(kf.split(all_train)):
  X_train, X_val = all_train[train_idx], all_train[test_idx]
  y_train, y_val = all_y[train_idx], all_y[test_idx]

  clf = make_pipeline(StandardScaler(), LogisticRegression(max_iter=10000))
  clf.fit(X_train, y_train)
  val_preds = clf.predict_proba(X_val)

  preds.append(clf.predict_proba(test_features)[:, 1])
  auc = roc_auc_score(y_val, val_preds[:, 1])
  history.append(auc)
  
  print("Fold: ", i, ": ", auc)
print(np.array(history).mean())

Fold:  0 :  0.9049040945026418
Fold:  1 :  0.8567538650188744
Fold:  2 :  0.8617260369910547
Fold:  3 :  0.874047314578005
Fold:  4 :  0.8618278758613752
0.8718518373903901


In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
kf = KFold(n_splits=5)

preds = []
history = []
for (i, (train_idx, test_idx)) in enumerate(kf.split(all_train)):
  X_train, X_val = all_train[train_idx], all_train[test_idx]
  y_train, y_val = all_y[train_idx], all_y[test_idx]

  clf = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=256, activation='tanh'))
  clf.fit(X_train, y_train)
  val_preds = clf.predict_proba(X_val)

  preds.append(clf.predict_proba(test_features)[:, 1])
  auc = roc_auc_score(y_val, val_preds[:, 1])
  history.append(auc)
  
  print("Fold: ", i, ": ", auc)

Fold:  0 :  0.9306995922541397
Fold:  1 :  0.9158291457286432
Fold:  2 :  0.918193104765543
Fold:  3 :  0.9171675191815858
Fold:  4 :  0.9076253709571954


# MLP 

In [ ]:
kf = KFold(n_splits=5)

preds = []
history = []
for (i, (train_idx, test_idx)) in enumerate(kf.split(all_train)):
  X_train, X_val = all_train[train_idx], all_train[test_idx]
  y_train, y_val = all_y[train_idx], all_y[test_idx]

  clf = make_pipeline(StandardScaler(), SVC(C = 10, gamma='auto', probability=True))
  clf.fit(X_train, y_train)
  val_preds = clf.predict_proba(X_val)

  preds.append(clf.predict_proba(test_features)[:, 1])
  auc = roc_auc_score(y_val, val_preds[:, 1])
  history.append(auc)
  
  print("Fold: ", i, ": ", auc)

Fold:  0 :  0.9678593216529492
Fold:  1 :  0.9362564821674251
Fold:  2 :  0.922313024740298
Fold:  3 :  0.906442336954159
Fold:  4 :  0.9137019010656956


In [ ]:
preds

[array([8.26891975e-01, 6.62909864e-03, 1.45318703e-02, ...,
        1.13724234e-05, 7.02646262e-04, 4.91667264e-02]),
 array([8.59687712e-01, 1.83992636e-03, 1.36919854e-02, ...,
        7.50487522e-05, 5.21598813e-04, 1.25965241e-02]),
 array([9.78598054e-01, 1.94842081e-02, 1.43094060e-02, ...,
        2.21298164e-06, 2.16038189e-03, 4.52193541e-03]),
 array([9.18012503e-01, 1.33576133e-02, 6.75130255e-03, ...,
        8.34337384e-05, 1.43515355e-04, 1.58712634e-01]),
 array([0.83079313, 0.00411595, 0.01699484, ..., 0.00108172, 0.00120131,
        0.04309534])]

In [ ]:
results = np.sum(preds, axis=0)/5
len(results)

1646

In [ ]:
result = pd.DataFrame()
result['id'] = test_df['id']
result['label'] = results

In [ ]:
result.to_csv('results.csv', index=False, header=None)

# Stacking part

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
all_train.shape, all_y.shape

((5172, 768), (5172,))

In [ ]:
X_strain, X_sval, y_strain, y_sval = train_test_split(all_train, all_y, test_size=0.3, shuffle=False)

In [ ]:
X_strain.shape

(3620, 768)

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(C = 10, gamma='auto', probability=True))
# clf = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=256, activation='tanh'))
clf.fit(X_strain, y_strain)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=10, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=True,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
y_sval_pred = clf.predict_proba(X_sval)[:, 1]
y_test_pred = clf.predict_proba(test_features)[:, 1]

In [ ]:
y_sval_pred

array([1.60105809e-05, 1.72222626e-03, 4.11168590e-06, ...,
       1.04663856e-04, 2.48527688e-06, 1.43420096e-01])

In [ ]:
pd.DataFrame(y_sval_pred, columns=["y_sval_pred_mlp"]).to_csv('y_sval_pred_mlp.csv', index=False)
pd.DataFrame(y_test_pred, columns=["y_test_pred_mlp"]).to_csv('y_test_pred_mlp.csv', index=False)

In [ ]:
y_sval

array([0, 0, 0, ..., 1, 1, 0])